## SegNet Segmentation Modelling
### RGB Image

In [1]:
import os
import random
from sklearn.model_selection import train_test_split
import shutil
from shutil import copyfile
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation, GlobalAveragePooling2D, Dense
from tensorflow.keras import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.preprocessing.image import array_to_img
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from PIL import Image
from keras.layers import Reshape
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Dice coefficient function
def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    y_true_flat = K.flatten(y_true)
    y_pred_flat = K.flatten(y_pred)

    y_true_flat = K.reshape(y_true_flat, K.shape(y_true))
    y_pred_flat = K.reshape(y_pred_flat, K.shape(y_pred))

    intersection = K.sum(y_true_flat * y_pred_flat)
    dice = (2.0 * intersection + smooth) / (K.sum(y_true_flat) + K.sum(y_pred_flat) + smooth)
    return dice

# Dice loss function
def dice_loss(y_true, y_pred):
    return 1.0 - dice_coefficient(y_true, y_pred)

# Intersection over Union (IoU) function
def iou(y_true, y_pred):
    y_true_shape = K.shape(y_true)
    y_pred_shape = K.shape(y_pred)

    if len(y_true_shape) == len(y_pred_shape):
        # Both tensors have the same number of dimensions
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
        union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) - intersection
    else:
        # One of the tensors has a different number of dimensions
        intersection = K.sum(K.abs(y_true * y_pred))
        union = K.sum(y_true) + K.sum(y_pred) - intersection

    iou = K.mean((intersection + 1e-15) / (union + 1e-15))
    return iou

In [12]:
def segnet(input_shape):
    # Encoder
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # Decoder
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    # Output
    outputs = Conv2D(3, (1, 1), activation='sigmoid', padding='same')(x)

    # Create the model
    model = Model(inputs=inputs, outputs=outputs)

    return model

In [13]:
input_shape = (256, 256, 3)
model = segnet(input_shape)

# model compile
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coefficient, iou])

In [14]:
model.input

<KerasTensor: shape=(None, 256, 256, 3) dtype=float32 (created by layer 'input_2')>

In [15]:
model.output

<KerasTensor: shape=(None, 256, 256, 3) dtype=float32 (created by layer 'conv2d_17')>

In [16]:
# Set the number of epochs and batch size for training
epochs = 30
batch_size = 4

output_folder = r'C:\Users\BoonJane\Desktop\oralcancer\output'

# Set the paths to the train, test, and validation data folders
train_data_dir = os.path.join(output_folder, 'train')
test_data_dir = os.path.join(output_folder, 'test')
val_data_dir = os.path.join(output_folder, 'val')

data_gen = ImageDataGenerator(rescale=1./255)

# Create the data generators for training, validation, and testing
train_data_gen = data_gen.flow_from_directory(
    train_data_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='input',  
    seed=42
)

val_data_gen = data_gen.flow_from_directory(
    val_data_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='input', 
    seed=42
)

test_data_gen = data_gen.flow_from_directory(
    test_data_dir,
    target_size=(256, 256),
    batch_size=batch_size,
    color_mode='rgb',
    class_mode='input', 
    seed=42
)

# Evaluate the model on the test dataset
for x, y_true in test_data_gen:
    y_pred = model.predict(x)
    break

Found 1056 images belonging to 4 classes.
Found 88 images belonging to 4 classes.
Found 88 images belonging to 4 classes.
1/1 [==============================] - 2s 2s/step


In [17]:
# check the shapes
print("Label data shape:", y_true.shape)
print("Predicted data shape:", y_pred.shape)

# check on data because IoU value is not logical
min_value_true = np.min(y_true)
max_value_true = np.max(y_true)
min_value_pred = np.min(y_pred)
max_value_pred = np.max(y_pred)

print("Min value of y_true:", min_value_true)
print("Max value of y_true:", max_value_true)
print("Min value of y_pred:", min_value_pred)
print("Max value of y_pred:", max_value_pred)

# # print y_true and y_predict
# print("y_true:", y_true)
# print("y_pred:", y_pred)

# check data type (need to be float)
print("Data type of y_true:", y_true.dtype)
print("Data type of y_pred:", y_pred.dtype)

Label data shape: (4, 256, 256, 3)
Predicted data shape: (4, 256, 256, 3)
Min value of y_true: 0.0
Max value of y_true: 1.0
Min value of y_pred: 0.48786575
Max value of y_pred: 0.50698286
Data type of y_true: float32
Data type of y_pred: float32


In [18]:
checkpoint_path = "segnetrgb_weights.h5"

checkpoint = ModelCheckpoint(
    checkpoint_path,
    monitor='val_loss', 
    verbose=1,  
    save_best_only=True,  
    save_weights_only=True  
)

# Train the model
history = model.fit(
    train_data_gen,
    steps_per_epoch=len(train_data_gen),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=len(val_data_gen),
    callbacks=[checkpoint],
)

Epoch 1/30
264/264 [==============================] - ETA: 0s - loss: 0.2528 - dice_coefficient: 0.7472 - iou: 0.6658
Epoch 1: val_loss improved from inf to 0.19332, saving model to segnetrgb_weights.h5
264/264 [==============================] - 921s 3s/step - loss: 0.2528 - dice_coefficient: 0.7472 - iou: 0.6658 - val_loss: 0.1933 - val_dice_coefficient: 0.8067 - val_iou: 0.7582
Epoch 2/30
264/264 [==============================] - ETA: 0s - loss: 0.1989 - dice_coefficient: 0.8011 - iou: 0.7517
Epoch 2: val_loss improved from 0.19332 to 0.19164, saving model to segnetrgb_weights.h5
264/264 [==============================] - 925s 4s/step - loss: 0.1989 - dice_coefficient: 0.8011 - iou: 0.7517 - val_loss: 0.1916 - val_dice_coefficient: 0.8084 - val_iou: 0.7430
Epoch 3/30
264/264 [==============================] - ETA: 0s - loss: 0.1916 - dice_coefficient: 0.8084 - iou: 0.7647
Epoch 3: val_loss improved from 0.19164 to 0.17308, saving model to segnetrgb_weights.h5
264/264 [==============

Epoch 24/30
264/264 [==============================] - ETA: 0s - loss: 0.1794 - dice_coefficient: 0.8206 - iou: 0.7838
Epoch 24: val_loss did not improve from 0.16325
264/264 [==============================] - 925s 4s/step - loss: 0.1794 - dice_coefficient: 0.8206 - iou: 0.7838 - val_loss: 0.1773 - val_dice_coefficient: 0.8227 - val_iou: 0.7869
Epoch 25/30
264/264 [==============================] - ETA: 0s - loss: 0.1790 - dice_coefficient: 0.8210 - iou: 0.7835
Epoch 25: val_loss improved from 0.16325 to 0.16252, saving model to segnetrgb_weights.h5
264/264 [==============================] - 921s 3s/step - loss: 0.1790 - dice_coefficient: 0.8210 - iou: 0.7835 - val_loss: 0.1625 - val_dice_coefficient: 0.8375 - val_iou: 0.7931
Epoch 26/30
264/264 [==============================] - ETA: 0s - loss: 0.1759 - dice_coefficient: 0.8241 - iou: 0.7835
Epoch 26: val_loss did not improve from 0.16252
264/264 [==============================] - 925s 4s/step - loss: 0.1759 - dice_coefficient: 0.8241

In [19]:
# Evaluate the model on the validation dataset
validation_loss, validation_dice_coefficient, validation_iou = model.evaluate(val_data_gen, steps=len(val_data_gen))
print("Validation Loss:", validation_loss)
print("Validation Dice Coefficient:", validation_dice_coefficient)
print("Validation IoU:", validation_iou)

22/22 [==============================] - 25s 1s/step - loss: 0.1744 - dice_coefficient: 0.8256 - iou: 0.7912
Validation Loss: 0.17441508173942566
Validation Dice Coefficient: 0.8255850076675415
Validation IoU: 0.7911903262138367


In [20]:
# Test the model on the test dataset
test_loss, test_dice_coefficient, test_iou = model.evaluate(test_data_gen, steps=len(test_data_gen))
print("Test Loss:", test_loss)
print("Test Dice Coefficient:", test_dice_coefficient)
print("Test IoU:", test_iou)

22/22 [==============================] - 24s 1s/step - loss: 0.1767 - dice_coefficient: 0.8233 - iou: 0.7913
Test Loss: 0.17667077481746674
Test Dice Coefficient: 0.8233292102813721
Test IoU: 0.7912797927856445


In [21]:
# Use the model for predictions
predictions = model.predict(test_data_gen)
# 'predictions' is an array containing the predicted masks for the test images

# Save the predicted masks as images
output_folder = r"C:\Users\BoonJane\Desktop\oralcancer\segnet\predictions"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for i, (x, _) in enumerate(test_data_gen):
    # Generate predictions for batch of images
    y_pred = model.predict(x)

    # Save each predicted mask as an image
    for j in range(len(y_pred)):
        mask = y_pred[j] > 0.5  # Threshold predicted mask
        mask_img = array_to_img(mask)
        mask_img.save(os.path.join(output_folder, f"prediction_{i * batch_size + j}.jpg"))

    if i == len(test_data_gen) - 1:
        # Processed all test data
        break

1/1 [==============================] - 1s 885ms/step


In [22]:
# Plot loss and metrics curves
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['dice_coefficient'], label='Training Dice Coefficient')
plt.plot(history.history['val_dice_coefficient'], label='Validation Dice Coefficient')
plt.plot(history.history['iou'], label='Training IoU')
plt.plot(history.history['val_iou'], label='Validation IoU')
plt.xlabel('Epochs')
plt.ylabel('Metrics')
plt.legend()

plt.tight_layout()
plt.show()

In [23]:
# save model weight
model.save('segnetrgb.h5')